## Custom vectorizer

In [19]:
import os
os.chdir('..')
print(os.getcwd())

/home/mono/materias/pln/repo


In [20]:
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pickle
import numpy as np

In [21]:
from sentiment.tass import InterTASSReader

# Train data
reader = InterTASSReader('corpus/InterTASS/PE/intertass-PE-train-tagged.xml')
tweets = list(reader.tweets())  # iterador sobre los tweets
X = list(reader.X())  # iterador sobre los contenidos de los tweets
y = list(reader.y())  # iterador sobre las polaridades de los tweets

# Development data
reader = InterTASSReader('corpus/InterTASS/PE/intertass-PE-development-tagged.xml')
tweets = list(reader.tweets())  # iterador sobre los tweets
X_dev = list(reader.X())  # iterador sobre los contenidos de los tweets
y_dev = list(reader.y())  # iterador sobre las polaridades de los tweets

In [6]:
# find tweet with string
for i, x in enumerate(X):
    if 'no' in x:
        break

print(i)
print(X[i])

0
Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.


### CountVectorizer as is

In [30]:
vect = CountVectorizer()
print(vect.fit_transform(X))


  (0, 1408)	1
  (0, 4049)	1
  (0, 1204)	1
  (0, 2551)	1
  (0, 3412)	1
  (0, 1482)	1
  (0, 1967)	1
  (0, 2760)	1
  (0, 2237)	1
  (0, 1735)	1
  (0, 4194)	2
  (0, 1452)	1
  (0, 3386)	1
  (0, 3943)	1
  (0, 3783)	1
  (0, 1754)	1
  (0, 2183)	1
  (0, 1046)	2
  (0, 1616)	1
  (0, 3750)	1
  (0, 3812)	1
  (1, 172)	1
  (1, 4295)	1
  (1, 4371)	1
  (1, 4084)	1
  :	:
  (997, 2855)	1
  (997, 4042)	1
  (997, 2926)	1
  (997, 867)	1
  (997, 2399)	1
  (997, 4089)	1
  (997, 657)	1
  (997, 818)	1
  (997, 2551)	1
  (997, 3386)	1
  (998, 1790)	1
  (998, 1788)	1
  (998, 4210)	1
  (998, 2774)	1
  (998, 1435)	1
  (998, 175)	1
  (998, 2237)	1
  (999, 968)	1
  (999, 1295)	1
  (999, 3727)	1
  (999, 3493)	1
  (999, 2388)	1
  (999, 2551)	2
  (999, 3386)	2
  (999, 1046)	1


## Changing the tokenizer

In [20]:
vect = CountVectorizer(tokenizer=word_tokenize)
vectorize = vect.build_analyzer()
print(vectorize(X[5]))

['@', 'fiorela_gue', 'aw', '!', 'recién', 'veo', 'esto', ',', 'sorrry', '!', 'tu', 'pues', 'me', 'dices', 'algo', 'y', 'luego', 'queda', 'en', 'nada', 'vamos', 'hoy', 'a', 'la', 'reu', 'de', 'mi', 'prima', '?', 'le', 'dije', 'a', 'mari']


In [20]:
def neg_handling_tokenizer(s, max_negations=3):
    neg_words = ['no', 'tampoco', 'ni']
    tokens = word_tokenize(s)
    make_opposite = False
    neg_count = 0
    for i in range(len(tokens)):
        if re.search(r'[.?\-",]+', tokens[i]):
            make_opposite = False
        if make_opposite and \
            tokens[i] not in neg_words and \
            neg_count <= max_negations:

            tokens[i] = 'NOT_' + tokens[i]
            neg_count += 1
        if tokens[i] in neg_words:
            make_opposite = True
            neg_count = 0
    return tokens

def build_tokenizer(max_negations=2):
    return lambda s: neg_handling_tokenizer(s, max_negations)

vect = CountVectorizer(tokenizer=build_tokenizer())
vectorize = vect.build_analyzer()
print(vectorize(X[10]))

['+saliste', 'ayer', '?', '-no', ',', 'me', 'quedé', 'en', 'casa', '.', '+qué', 'aburrido', 'te', 'has', 'vuelto', '!', '-eso', 'no', 'NOT_dices', 'NOT_cuando', 'NOT_llega', 'la', 'quincena', 'y', 'no', 'NOT_tienes', 'ni', 'NOT_para', 'NOT_el', 'NOT_pasaje', '.']


## Changing the preprocessor

In [32]:
def my_preprocessor(doc):
    tagged_users_pat = r'@\w+'
    urls_pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    vocals_rep_pat = r'(\w)\1{1,}'
    laugh_pat = r'\b[jah]+\b'
    return re.sub(tagged_users_pat, '',
           re.sub(laugh_pat, '<LAUGH>',
           re.sub(urls_pat, '',
           re.sub(vocals_rep_pat, '\g<1>',
           doc.lower()))))

vect = CountVectorizer(preprocessor=my_preprocessor)
vectorize = vect.build_analyzer()
print(vectorize(X[1]))

['ayer', 'preguntaban', 'dónde', 'están', 'las', 'solteras', 'todo', 'mi', 'grupo', 'alza', 'la', 'mano', 'yo', 'la', 'única', 'que', 'no', 'todas', 'voltean', 'LAUGH', 'verme', 'LAUGH']


## Analyzing the models

### Train

In [28]:
!python sentiment/scripts/train.py -m clf -c maxent

/home/mono/materias/pln/pln-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mono/materias/pln/pln-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


### Maxent features

In [29]:
with open('models/ES-maxent.model', 'rb') as fp:
    model = pickle.load(fp)
        
y_pred = model.predict(X_dev)
y_pred_proba = model._pipeline.predict_proba(X_dev)

from sentiment.analysis import print_maxent_features, print_feature_weights_for_item

vect = model._pipeline.named_steps['vect']
clf = model._pipeline.named_steps['clf']
print_maxent_features(vect, clf)

N:
	! buena gracias genial gran ([-1.11747036 -0.70254198 -0.64637816 -0.64238543 -0.64020784])
	peor odio ni no triste ([0.70857828 0.89127316 0.9378617  1.05487194 1.15167997])
NEU:
	hoy gracias # hacer cosas ([-0.91044231 -0.87210966 -0.86211886 -0.75393938 -0.62556682])
	vez sido aunque nerviosa felices ([0.72865744 0.75207712 0.78020148 0.82442367 0.83693381])
NONE:
	mal ... buen no ser ([-0.84558229 -0.7118646  -0.70027885 -0.67418331 -0.67183291])
	alguna jugar vídeo semana ? ([0.72483241 0.81335749 0.88829897 0.96288195 1.34383161])
P:
	no triste ni odio vuelta ([-0.77035651 -0.75227158 -0.61878566 -0.49820698 -0.49676044])
	guapa genial gracias buen ! ([0.96675182 0.99257572 1.06841277 1.24693733 1.32596686])


### Sorting wrongly tagged tweets

In [16]:
classes = model._pipeline.classes_
print(classes)
wrongly_tagged = [(X_dev[i], y_dev[i], y_pred[i], y_pred_proba[i], i) 
                  for i in range(len(X_dev)) if y_pred[i] != y_dev[i]]

""" Sort by prob difference"""
b = [t for t in sorted(wrongly_tagged,
       key=lambda t:
           t[3][np.where(classes == t[1])][0] -
           t[3][np.where(classes == t[2])][0])]
print(b)

['N' 'NEU' 'NONE' 'P']
[('Buen día.... Para delante una nueva semana con fe...!!!! Con fe ... Cuanto tiempo me aguantas con mi forma de ser...', 'NONE', 'P', array([0.04669929, 0.0314912 , 0.0202422 , 0.90156731]), 76), ('Buen año nuevo gracias.... @alesalazarm @marielaslazar @romi_rb5 @AnikaJohana #conlasmejores', 'NONE', 'P', array([0.09619563, 0.02283504, 0.02643733, 0.854532  ]), 257), ('Y comenzó el día mas amado y esperado ja ja! "Lunes" a comenzar con los dos pies bien puestos en la tierra. Así que adelante!!.', 'N', 'P', array([0.07311384, 0.01840572, 0.02999888, 0.87848156]), 6), ('En pocos minutos empieza Combate!! Usa el HT #CbtHastaSiempre y coméntanos que fue lo que más te gusto de tu programa favorito...', 'NONE', 'P', array([0.05242683, 0.06869499, 0.04962524, 0.82925294]), 137), ('!Feliz año nuevo! este 2017 sea realmente un año muy lindo y maravilloso ^_^', 'NONE', 'P', array([0.04649688, 0.07665219, 0.05610333, 0.8207476 ]), 201), ('@JEESrcismo por eso mismo. Yo no ve

### Modifying the tweet

In [25]:
tweet = X_dev[76]
print(model._pipeline.classes_, y_dev[76], y_pred[76])
print(tweet)
print(model.predict_proba([tweet]))

new_tweet = 'Otro día. Aunque para delante alguna semana con fe. ' \
            'Cuanto tiempo me aguantas con mi forma de ser.'
print(model.predict_proba([new_tweet]))
print(model.predict([new_tweet]))

new_tweet = ''
print(model.predict_proba([new_tweet]))
print(model.predict([new_tweet]))


['N' 'NEU' 'NONE' 'P'] NONE P
Buen día.... Para delante una nueva semana con fe...!!!! Con fe ... Cuanto tiempo me aguantas con mi forma de ser...
[[0.04669929 0.0314912  0.0202422  0.90156731]]
[[0.21668909 0.16951191 0.25684346 0.35695554]]
['P']
[[0.29837558 0.24831629 0.23030293 0.22300519]]
['N']


## Sentiment polarity lexicon

In [47]:
from sentiment import word_polarities

tweet = X_dev[76]
counter = word_polarities.PolarizedWordsCounter()
print(tweet)
print(counter.get_feature_names())
print(counter.transform([tweet]))

print(counter.classify(word_polarities._tokenize(tweet)))

# Change thresholds and re-classify
counter = word_polarities.PolarizedWordsCounter(thresholds=[1.5, 2.5])
print(counter.transform([tweet]))
print(counter.classify(word_polarities._tokenize(tweet)))


Buen día.... Para delante una nueva semana con fe...!!!! Con fe ... Cuanto tiempo me aguantas con mi forma de ser...
['neg', 'neu', 'pos']
[[0, 5, 2]]
{'neg': [], 'pos': ['día', 'ser'], 'neu': ['delante', 'semana', 'cuanto', 'tiempo', 'forma']}
[[0, 6, 1]]
{'neg': [], 'pos': ['ser'], 'neu': ['día', 'delante', 'semana', 'cuanto', 'tiempo', 'forma']}
